## Modelo Churn Forecast

In [1]:
#importamos los paquetes necesarios
import cx_Oracle
import pandas as pd
import numpy as np
import openpyxl

In [2]:
cx_Oracle.init_oracle_client(lib_dir='C:/instantclient_21_9') 
dsn_tns = cx_Oracle.makedsn('bd-dwh.avantel.com.co', '1521', service_name='DWHWOM') # if needed, place an 'r' before any parameter in order to address special characters such as '\'.
conn = cx_Oracle.connect(user=r'ANALITICA_WOM', password='ANALI2023*5', dsn=dsn_tns)  

In [3]:
mes_inicio = 202301
mes_fin = 202303

In [ ]:
#Importamos los planes y sus homologaciones para ser usadas en el script de lectura
plan_homol = pd.read_excel('planes_homologacion.xlsx',sheet_name='Planes')

#Tomamos la clasif 3 que posee la clasificación dada por pricing
plan_homol_script_t = []
for index, row in plan_homol.iterrows():
    if index + 1 < plan_homol.shape[0]:
        #creamos el script para crear la tabla de homologación para el siguiente paso en sql
        homol_script_u = f"SELECT '{row['Plan']}','{row['Clasif 3'][1:]}' FROM dual UNION ALL"
    else:
        #creamos el script para crear la tabla de homologación para el siguiente paso en sql sin el Union ALL
        homol_script_u = f"SELECT '{row['Plan']}','{row['Clasif 3'][1:]}' FROM dual"

    plan_homol_script_t.append(homol_script_u)

plan_homol_script = '\n'.join(plan_homol_script_t)

In [4]:
CHURN_QRY = f"""
WITH PLAN_MAP (PLAN_NOMBRE, PLAN_NOMBRE2) AS (
                    {plan_homol_script} 
                    )
CHURNERS AS (
    SELECT /*+PARALLEL(8)*/
        SUBSCRIBER_ID,
        PERIODO_PROCESO_CODIGO,
        MOVIMIENTO_TIPO_NOMBRE,
        FECHA_ALTA
    FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
    WHERE subscriber_id NOT IN (SELECT subscriber_id FROM WOM_AA_USUARIOS_FRAUDE)
        AND PERIODO_PROCESO_CODIGO >= {mes_inicio}
        AND PERIODO_PROCESO_CODIGO <= {mes_fin}
        AND SERVICIO = 'Postpaid'
        AND MOVIMIENTO_NOMBRE = 'DESCONEXION'
), 
CHURN_DESC AS (
    SELECT /*+PARALLEL(8)*/
        CHU.PERIODO_PROCESO_CODIGO,
        TO_CHAR(MONTHS_BETWEEN(TO_DATE(CHU.PERIODO_PROCESO_CODIGO, 'YYYYMM'),TRUNC(ENDI.FECHA_ALTA,'MM'))) ANTIGUEDAD_MESES,
        ROW_NUMBER() OVER (PARTITION BY ENDI.SUBSCRIBER_ID ORDER BY ENDI.PERIODO_PROCESO_CODIGO DESC) RANK_PERIODO,
        CHU.SUBSCRIBER_ID,
        CHU.MOVIMIENTO_TIPO_NOMBRE,
        ENDI.MOVIMIENTO_TIPO_NOMBRE MOVIMIENTO_ORIGEN,
        ENDI.OPERADOR_PORTA_ORIGEN,
        PLA.PLAN_TIPO_COMERCIAL,
        PLA.PLAN_NOMBRE PLAN_NOMBRE_NH,
        CONS.CONSULTOR_CANAL_NOMBRE,
        CONS.CONSULTOR_REGIONAL_NOMBRE,
        LEADS.CATEGORIA_BOUND
    FROM CHURNERS CHU
    LEFT JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING ENDI ON CHU.SUBSCRIBER_ID = ENDI.SUBSCRIBER_ID AND CHU.FECHA_ALTA = ENDI.FECHA_ALTA AND ENDI.MOVIMIENTO_NOMBRE = 'ACTIVACION'
    LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES PLA ON ENDI.PLAN_COMERCIAL_DK = PLA.PLAN_COMERCIAL_DK
    LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES CONS ON ENDI.CONSULTOR_DK = CONS.CONSULTOR_DK
    LEFT JOIN temp_categoria_mkt_v2 LEADS ON CHU.SUBSCRIBER_ID = LEADS.SUBSCRIBER_ID AND CHU.PERIODO_PROCESO_CODIGO=LEADS.PERIODO_PROCESO_CODIGO

),

CHURN_LIMP AS (
    SELECT /*+PARALLEL(8)*/
        PERIODO_PROCESO_CODIGO,
        MOVIMIENTO_ORIGEN,
        ANTIGUEDAD_MESES,
        CASE 
            WHEN ANTIGUEDAD_MESES <=3 THEN 'NUEVO'
            WHEN ANTIGUEDAD_MESES <=6 THEN 'MEDIO'
            WHEN ANTIGUEDAD_MESES <=12 THEN 'ANTIGUO'
        ELSE 'MUY ANTIGUO' END AS ANTIGUEDAD_AGRUP,
        PLAN_TIPO_COMERCIAL,
        PLAN_NOMBRE_NH,
        CASE 
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN S%' THEN 'S'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN M%' THEN 'M'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% S%' THEN 'S'   
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% M%' THEN 'M'        
        ELSE 'OTROS' END AS PLAN_NOMBRE_H,
        
        CONSULTOR_CANAL_NOMBRE,
        CASE 
            WHEN CONSULTOR_CANAL_NOMBRE ='Tiendas' THEN 'Shops'
            WHEN CONSULTOR_CANAL_NOMBRE ='Fuerza De Venta Indirecta' THEN 'Dealers'
            WHEN CONSULTOR_CANAL_NOMBRE ='Outbound' THEN 'Outbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Inbound' THEN 'Inbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Kioscos' THEN 'Kiosks'
            WHEN CONSULTOR_CANAL_NOMBRE ='Ventas Online' THEN 'eCommerce'
        ELSE 'OTROS' END AS CONSULTOR_CANAL_NOMBRE_H,
        CASE WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Outbound' THEN 'Outbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Inbound' THEN 'Inbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Ventas Online' THEN 'No Asistidos'
        WHEN CATEGORIA_BOUND IS NULL THEN 'OTROS'
        ELSE CATEGORIA_BOUND END AS CATEGORIA_BOUND,
        MOVIMIENTO_TIPO_NOMBRE,
        CASE
            WHEN OPERADOR_PORTA_ORIGEN = 'TELEFONICA' THEN 'MOVISTAR'
            WHEN OPERADOR_PORTA_ORIGEN = 'COMCEL' THEN 'CLARO'
            WHEN OPERADOR_PORTA_ORIGEN = 'TIGO' THEN 'TIGO'
            WHEN OPERADOR_PORTA_ORIGEN IS NULL THEN 'LINEA NUEVA'
        ELSE 'OTROS' END OPERADOR_PORTA_ORIGEN,
        CASE 
            WHEN CONSULTOR_REGIONAL_NOMBRE IN ('-1','-2') THEN 'Regional Centro' 
            WHEN CONSULTOR_REGIONAL_NOMBRE IS NULL THEN 'Regional Centro'
        ELSE CONSULTOR_REGIONAL_NOMBRE END REGIONAL
    FROM CHURN_DESC
    
)

SELECT /*+PARALLEL(8)*/
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_NH,
    PLAN_NOMBRE_H,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL,
    COUNT(*) SUBS
FROM CHURN_LIMP
GROUP BY 
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_NH,
    PLAN_NOMBRE_H,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL
"""

In [5]:
dfl = []  

for chunk in pd.read_sql(CHURN_QRY, con=conn ,chunksize=10000):
    dfl.append(chunk)
    
churn_df = pd.concat(dfl, ignore_index=True)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
churn_df [churn_df['CATEGORIA_BOUND'].isnull()]

,PERIODO_PROCESO_CODIGO,MOVIMIENTO_ORIGEN,ANTIGUEDAD_AGRUP,ANTIGUEDAD_MESES,PLAN_TIPO_COMERCIAL,PLAN_NOMBRE_NH,PLAN_NOMBRE_H,CONSULTOR_CANAL_NOMBRE,CONSULTOR_CANAL_NOMBRE_H,CATEGORIA_BOUND,MOVIMIENTO_TIPO_NOMBRE,OPERADOR_PORTA_ORIGEN,REGIONAL,SUBS


In [7]:
churn_df.groupby(['CONSULTOR_CANAL_NOMBRE','CATEGORIA_BOUND']).agg(
                                          {
                                           'SUBS':['count','sum']
                                          })

SUBS        
                                                        count     sum
CONSULTOR_CANAL_NOMBRE        CATEGORIA_BOUND                        
-1                            OTROS                        11      12
-2                            OTROS                        97     121
Cadenas                       OTROS                      4549   15056
Convenios Remotos             OTROS                         6       6
Distribuidores B2B e Internet OTROS                         6      10
E-Dealers                     OTROS                      1999    4389
Escuadron Negocios            OTROS                        48     101
External organization         OTROS                       351     433
Fuerza De Venta Indirecta     OTROS                     10412   44149
Inbound                       Inbound                    4668   25523
                              Remotos Asistidos Miguel    689    1011
Internal organization         OTROS                       118     239
Kioscos                       OTROS                      8460   25442
Outbound                      Outbound                   6201   62864
                              Remotos Asistidos Miguel    684     959
Self-employed                 OTROS                         4       4
Tiendas                       OTROS                     18092  122118
Tiendas Express               OTROS                      4908   16173
Tiendas Movil                 OTROS                        10      19
Tiendas TuWOM                 OTROS                       802    1419
Vending Machine               OTROS                         6       6
Ventas Online                 No Asistidos                436     771

In [ ]:
churn_df.groupby(['CONSULTOR_CANAL_NOMBRE','CATEGORIA_BOUND']).sum('SUBS')

In [ ]:
churn_df

In [8]:
GROSS_QRY = f"""
WITH 
GROSS AS 
(
    SELECT 
        'GROSS' MOVIMIENTO_TIPO_NOMBRE,
        ENDI.SUBSCRIBER_ID,
        ENDI.PERIODO_PROCESO_CODIGO,
        TO_CHAR(MONTHS_BETWEEN(TO_DATE(ENDI.PERIODO_PROCESO_CODIGO, 'YYYYMM'),TRUNC(ENDI.FECHA_ALTA,'MM'))) ANTIGUEDAD_MESES,
        ENDI.MOVIMIENTO_TIPO_NOMBRE MOVIMIENTO_ORIGEN,
        ENDI.OPERADOR_PORTA_ORIGEN,
        PLA.PLAN_TIPO_COMERCIAL,
        PLA.PLAN_NOMBRE PLAN_NOMBRE_NH,
        CONS.CONSULTOR_CANAL_NOMBRE,
        CONS.CONSULTOR_REGIONAL_NOMBRE,
        LEADS.CATEGORIA_BOUND
    FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING ENDI
    LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES PLA ON ENDI.PLAN_COMERCIAL_dk = PLA.plan_COMERCIAL_dk
    LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES CONS ON ENDI.CONSULTOR_DK = CONS.CONSULTOR_DK
    LEFT JOIN temp_categoria_mkt_v2 LEADS ON ENDI.SUBSCRIBER_ID = LEADS.SUBSCRIBER_ID AND ENDI.PERIODO_PROCESO_CODIGO=LEADS.PERIODO_PROCESO_CODIGO
    WHERE 
    ENDI.MOVIMIENTO_NOMBRE = 'ACTIVACION'
    AND ENDI.SERVICIO = 'Postpaid'
    AND ENDI.PERIODO_PROCESO_CODIGO >= {mes_inicio}
    AND ENDI.PERIODO_PROCESO_CODIGO <= {mes_fin}
    AND ENDI.subscriber_id NOT IN (SELECT subscriber_id FROM WOM_AA_USUARIOS_FRAUDE)
    

),

GROSS_LIMP AS (
    SELECT /*+PARALLEL(8)*/
        PERIODO_PROCESO_CODIGO,
        MOVIMIENTO_ORIGEN,
        ANTIGUEDAD_MESES,
        CASE 
            WHEN ANTIGUEDAD_MESES <=3 THEN 'NUEVO'
            WHEN ANTIGUEDAD_MESES <=6 THEN 'MEDIO'
            WHEN ANTIGUEDAD_MESES <=12 THEN 'ANTIGUO'
        ELSE 'MUY ANTIGUO' END AS ANTIGUEDAD_AGRUP,
        PLAN_TIPO_COMERCIAL,
        PLAN_NOMBRE_NH,
        CASE 
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN S%' THEN 'S'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN M%' THEN 'M'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% S%' THEN 'S'   
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% M%' THEN 'M'        
        ELSE 'OTROS' END AS PLAN_NOMBRE_H,
        CONSULTOR_CANAL_NOMBRE,
        CASE 
            WHEN CONSULTOR_CANAL_NOMBRE ='Tiendas' THEN 'Shops'
            WHEN CONSULTOR_CANAL_NOMBRE ='Fuerza De Venta Indirecta' THEN 'Dealers'
            WHEN CONSULTOR_CANAL_NOMBRE ='Outbound' THEN 'Outbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Inbound' THEN 'Inbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Kioscos' THEN 'Kiosks'
            WHEN CONSULTOR_CANAL_NOMBRE ='Ventas Online' THEN 'eCommerce'
        ELSE 'OTROS' END AS CONSULTOR_CANAL_NOMBRE_H,
        MOVIMIENTO_TIPO_NOMBRE,
        CASE WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Outbound' THEN 'Outbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Inbound' THEN 'Inbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Ventas Online' THEN 'No Asistidos'
        WHEN CATEGORIA_BOUND IS NULL THEN 'OTROS'
        ELSE CATEGORIA_BOUND END AS CATEGORIA_BOUND,
        CASE
            WHEN OPERADOR_PORTA_ORIGEN = 'TELEFONICA' THEN 'MOVISTAR'
            WHEN OPERADOR_PORTA_ORIGEN = 'COMCEL' THEN 'CLARO'
            WHEN OPERADOR_PORTA_ORIGEN = 'TIGO' THEN 'TIGO'
            WHEN OPERADOR_PORTA_ORIGEN IS NULL THEN 'LINEA NUEVA'
        ELSE 'OTROS' END OPERADOR_PORTA_ORIGEN,
        CASE 
            WHEN CONSULTOR_REGIONAL_NOMBRE IN ('-1','-2') THEN 'Regional Centro' 
            WHEN CONSULTOR_REGIONAL_NOMBRE IS NULL THEN 'Regional Centro'
        ELSE CONSULTOR_REGIONAL_NOMBRE END REGIONAL
    FROM GROSS
)

SELECT /*+PARALLEL(8)*/
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_H,
    PLAN_NOMBRE_NH,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL,
    COUNT(*) SUBS
FROM GROSS_LIMP
GROUP BY 
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_H,
    PLAN_NOMBRE_NH,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL
"""

In [9]:
dfl = []  

for chunk in pd.read_sql(GROSS_QRY, con=conn ,chunksize=10000):
    dfl.append(chunk)
    
gross_df = pd.concat(dfl, ignore_index=True)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
gross_df.groupby(['PERIODO_PROCESO_CODIGO']).sum('SUBS')

,SUBS
PERIODO_PROCESO_CODIGO,
202301,166446
202302,153770
202303,165499


In [11]:
gross_df

,PERIODO_PROCESO_CODIGO,MOVIMIENTO_ORIGEN,ANTIGUEDAD_AGRUP,ANTIGUEDAD_MESES,PLAN_TIPO_COMERCIAL,PLAN_NOMBRE_H,PLAN_NOMBRE_NH,CONSULTOR_CANAL_NOMBRE,CONSULTOR_CANAL_NOMBRE_H,CATEGORIA_BOUND,MOVIMIENTO_TIPO_NOMBRE,OPERADOR_PORTA_ORIGEN,REGIONAL,SUBS
0,202302,GROSS ADDS PORTACION,NUEVO,0,Mobile,L,Plan L 90_GB/ilim_Min y SMS,Inbound,Inbound,Inbound,GROSS,MOVISTAR,Regional Centro,141
1,202301,GROSS ADDS MIGRACION,NUEVO,0,Mobile,M,Plan M3 40_GB/ili_MIN/ili_SMS,Inbound,Inbound,Inbound,GROSS,LINEA NUEVA,Regional Centro,244
2,202301,GROSS ADDS NUEVA,NUEVO,0,BroadBand,XL,Internet Móvil XL3 120_GB/full_VEL,Kioscos,Kiosks,OTROS,GROSS,LINEA NUEVA,Regional Centro,76
3,202301,GROSS ADDS MIGRACION,NUEVO,0,Mobile,S,Plan S3 Plus 30_GB/ili_MIN/ili_SMS,Kioscos,Kiosks,OTROS,GROSS,TIGO,Regional Centro,5
4,202302,GROSS ADDS PORTACION,NUEVO,0,Mobile,S,Plan S 40_GB/ilim_Min y SMS,Tiendas,Shops,OTROS,GROSS,OTROS,Regional Oriente,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7210,202301,GROSS ADDS MIGRACION,NUEVO,0,Mobile,S,Plan S3 Plus 30_GB/ili_MIN/ili_SMS,Cadenas,OTROS,OTROS,GROSS,CLARO,Regional Oriente,1
7211,202301,GROSS ADDS PORTACION,NUEVO,0,BroadBand,XL,Internet Móvil XL3 120_GB/full_VEL,Kioscos,Kiosks,OTROS,GROSS,TIGO,Regional Centro,1
7212,202302,GROSS ADDS NUEVA,NUEVO,0,B2B,S,Plan NEGOCIOS S 35_GB/ili_MIN/ili_SMS,Kioscos,Kiosks,OTROS,GROSS,LINEA NUEVA,Regional Costa,2
7213,202301,GROSS ADDS PORTACION,NUEVO,0,Mobile,S,Plan S Plus 25_GB/ili_MIN/ili_SMS,Tiendas,Shops,OTROS,GROSS,CLARO,Regional Occidente,1


In [12]:
PERMANECE_QRY = f"""
WITH   
PERM AS (
    SELECT 
        SUBSCRIBER_ID,
        PERIODO_PROCESO_CODIGO,
        FECHA_ALTA,
        MOVIMIENTO_TIPO_NOMBRE
    FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
    WHERE PERIODO_PROCESO_CODIGO >= {mes_inicio}
        AND PERIODO_PROCESO_CODIGO <= {mes_fin}
        AND SERVICIO = 'Postpaid'
        AND MOVIMIENTO_NOMBRE = 'PERMANECE'
        and subscriber_id NOT IN (SELECT subscriber_id FROM WOM_AA_USUARIOS_FRAUDE)
),

PERM_DESC AS (
SELECT 
    PERM.PERIODO_PROCESO_CODIGO,
    TO_CHAR(MONTHS_BETWEEN(TO_DATE(PERM.PERIODO_PROCESO_CODIGO, 'YYYYMM'),TRUNC(ENDI.FECHA_ALTA,'MM'))) ANTIGUEDAD_MESES,
    ROW_NUMBER() OVER (PARTITION BY ENDI.SUBSCRIBER_ID, PERM.PERIODO_PROCESO_CODIGO ORDER BY ENDI.PERIODO_PROCESO_CODIGO DESC) RANK_PERIODO,
    PERM.SUBSCRIBER_ID,
    PERM.MOVIMIENTO_TIPO_NOMBRE,
    ENDI.MOVIMIENTO_TIPO_NOMBRE MOVIMIENTO_ORIGEN,
    ENDI.OPERADOR_PORTA_ORIGEN,
    PLA.PLAN_TIPO_COMERCIAL,
    PLA.PLAN_NOMBRE PLAN_NOMBRE_NH,
    CONS.CONSULTOR_CANAL_NOMBRE,
    CONS.CONSULTOR_REGIONAL_NOMBRE,
    LEADS.CATEGORIA_BOUND
FROM PERM PERM
LEFT JOIN DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING ENDI 
    ON PERM.SUBSCRIBER_ID = ENDI.SUBSCRIBER_ID AND PERM.FECHA_ALTA = ENDI.FECHA_ALTA
    AND ENDI.MOVIMIENTO_NOMBRE = 'ACTIVACION'
LEFT JOIN DWH_BODEGA_WOM.DIM_PLANES PLA ON ENDI.PLAN_COMERCIAL_DK = PLA.PLAN_COMERCIAL_DK
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES CONS ON ENDI.CONSULTOR_DK = CONS.CONSULTOR_DK
LEFT JOIN temp_categoria_mkt_v2 LEADS ON ENDI.SUBSCRIBER_ID = LEADS.SUBSCRIBER_ID AND ENDI.PERIODO_PROCESO_CODIGO=LEADS.PERIODO_PROCESO_CODIGO

),

PERMANECE_LIMP AS (
    SELECT /*+PARALLEL(8)*/
        PERIODO_PROCESO_CODIGO,
        MOVIMIENTO_ORIGEN,
        ANTIGUEDAD_MESES,
        CASE 
            WHEN ANTIGUEDAD_MESES <=3 THEN 'NUEVO'
            WHEN ANTIGUEDAD_MESES <=6 THEN 'MEDIO'
            WHEN ANTIGUEDAD_MESES <=12 THEN 'ANTIGUO'
        ELSE 'MUY ANTIGUO' END AS ANTIGUEDAD_AGRUP,
        PLAN_TIPO_COMERCIAL,
        PLAN_NOMBRE_NH,
        CASE 
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN S%' THEN 'S'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN M%' THEN 'M'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '%PLAN XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XS%' THEN 'XS'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% S%' THEN 'S'   
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% L%' THEN 'L'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% XL%' THEN 'XL'
            WHEN UPPER(PLAN_NOMBRE_NH) LIKE '% M%' THEN 'M'        
        ELSE 'OTROS' END AS PLAN_NOMBRE_H,

        CONSULTOR_CANAL_NOMBRE,
        
        CASE 
            WHEN CONSULTOR_CANAL_NOMBRE ='Tiendas' THEN 'Shops'
            WHEN CONSULTOR_CANAL_NOMBRE ='Fuerza De Venta Indirecta' THEN 'Dealers'
            WHEN CONSULTOR_CANAL_NOMBRE ='Outbound' THEN 'Outbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Inbound' THEN 'Inbound'
            WHEN CONSULTOR_CANAL_NOMBRE ='Kioscos' THEN 'Kiosks'
            WHEN CONSULTOR_CANAL_NOMBRE ='Ventas Online' THEN 'eCommerce'
        ELSE 'OTROS' END AS CONSULTOR_CANAL_NOMBRE_H,
        CASE WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Outbound' THEN 'Outbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Inbound' THEN 'Inbound'
        WHEN CATEGORIA_BOUND IS NULL AND CONSULTOR_CANAL_NOMBRE = 'Ventas Online' THEN 'No Asistidos'
        WHEN CATEGORIA_BOUND IS NULL THEN 'OTROS'
        ELSE CATEGORIA_BOUND END AS CATEGORIA_BOUND,
        MOVIMIENTO_TIPO_NOMBRE,
        CASE
            WHEN OPERADOR_PORTA_ORIGEN = 'TELEFONICA' THEN 'MOVISTAR'
            WHEN OPERADOR_PORTA_ORIGEN = 'COMCEL' THEN 'CLARO'
            WHEN OPERADOR_PORTA_ORIGEN = 'TIGO' THEN 'TIGO'
            WHEN OPERADOR_PORTA_ORIGEN IS NULL THEN 'LINEA NUEVA'
        ELSE 'OTROS' END OPERADOR_PORTA_ORIGEN,
        CASE 
            WHEN CONSULTOR_REGIONAL_NOMBRE IN ('-1','-2') THEN 'Regional Centro' 
            WHEN CONSULTOR_REGIONAL_NOMBRE IS NULL THEN 'Regional Centro'
        ELSE CONSULTOR_REGIONAL_NOMBRE END REGIONAL
    FROM PERM_DESC
)

SELECT /*+PARALLEL(8)*/
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_NH,
    PLAN_NOMBRE_H,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL,
    COUNT(*) SUBS
FROM PERMANECE_LIMP
GROUP BY 
    PERIODO_PROCESO_CODIGO,
    MOVIMIENTO_ORIGEN,
    ANTIGUEDAD_AGRUP,
    ANTIGUEDAD_MESES,
    PLAN_TIPO_COMERCIAL,
    PLAN_NOMBRE_NH,
    PLAN_NOMBRE_H,
    CONSULTOR_CANAL_NOMBRE,
    CONSULTOR_CANAL_NOMBRE_H,
    CATEGORIA_BOUND,
    MOVIMIENTO_TIPO_NOMBRE,
    OPERADOR_PORTA_ORIGEN,
    REGIONAL
"""

In [13]:
%%time
dfl = []  

for chunk in pd.read_sql(PERMANECE_QRY, con=conn ,chunksize=10000):
    dfl.append(chunk)
    
permanece_df = pd.concat(dfl, ignore_index=True)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Wall time: 1min


In [14]:
#Comprobar que el gross corresponda con lo ejecutado total
churn_df.groupby(['PERIODO_PROCESO_CODIGO']).sum('SUBS')
#1433226

,SUBS
PERIODO_PROCESO_CODIGO,
202301,104104
202302,104195
202303,112532


In [15]:
#Comprobar que el churn corresponda con lo ejecutado total
churn_df.groupby(['PLAN_NOMBRE_NH']).sum()['SUBS']

PLAN_NOMBRE_NH
#Int Móvil Plus M 90_GB/full_VEL/65_COM        5
#Int Móvil Plus XL ilim_GB/full_VEL            4
#Int Móvil Plus XL ilimi_GB/full_VEL           1
#Internet Móvil L 80_GB/full_VEL              12
#Internet Móvil L 80_GB/full_VEL_              2
                                           ...  
Plan XL ili_GB/ili_MIN/ili_SMS              3972
Plan XL ilim_GB/ilim_Min y SMS              1309
Plan XL3 ili_GB/ili_MIN/ili_SMS            20322
Plan XS 15_GB/ili_MIN/ili_SMS               3820
Plan XS 18_GB/ilim_Min y SMS                 324
Name: SUBS, Length: 107, dtype: int64

In [16]:
#creamos copias de los dfs
churn_df_t = churn_df.copy()
permanece_df_t = permanece_df.copy()
gross_df_t = gross_df.copy()

In [ ]:
#Las antiguedades None se trasnforman en 1 para poder hacer los calculos correspondientes
#churn_df_t['ANTIGUEDAD_MESES'] = churn_df_t['ANTIGUEDAD_MESES'].fillna('1')
#permanece_df_t['ANTIGUEDAD_MESES'] = permanece_df_t['ANTIGUEDAD_MESES'].fillna('1')

In [17]:
#columnas a utilizar
cols_select = ['PERIODO_PROCESO_CODIGO', 'MOVIMIENTO_ORIGEN', 'ANTIGUEDAD_AGRUP', 'ANTIGUEDAD_MESES', 'PLAN_TIPO_COMERCIAL','PLAN_NOMBRE_H', 'PLAN_NOMBRE_NH', 'CONSULTOR_CANAL_NOMBRE','CONSULTOR_CANAL_NOMBRE_H', 'CATEGORIA_BOUND',
              'OPERADOR_PORTA_ORIGEN', 'REGIONAL', 'SUBS','MOVIMIENTO_TIPO_NOMBRE']


#columnas para generar la tabla pivot
cols_unpivot = ['PERIODO_PROCESO_CODIGO', 'MOVIMIENTO_ORIGEN', 'ANTIGUEDAD_AGRUP', 'ANTIGUEDAD_MESES', 'PLAN_TIPO_COMERCIAL', 'PLAN_NOMBRE_H', 'PLAN_NOMBRE_NH', 'CONSULTOR_CANAL_NOMBRE','CONSULTOR_CANAL_NOMBRE_H', 'CATEGORIA_BOUND',
              'OPERADOR_PORTA_ORIGEN', 'REGIONAL']

#concat de tablas
df = pd.concat([churn_df[cols_select], gross_df[cols_select], permanece_df[cols_select]]).reset_index(drop=True)

#el fill sirve para no desechar SUBS de consultas que no traen información de gross 
df = df.fillna('NULL').groupby(['PERIODO_PROCESO_CODIGO', 'MOVIMIENTO_ORIGEN', 'ANTIGUEDAD_AGRUP', 'ANTIGUEDAD_MESES', 'PLAN_TIPO_COMERCIAL','PLAN_NOMBRE_H', 'PLAN_NOMBRE_NH', 'CONSULTOR_CANAL_NOMBRE' ,'CONSULTOR_CANAL_NOMBRE_H','CATEGORIA_BOUND',
              'OPERADOR_PORTA_ORIGEN', 'REGIONAL','MOVIMIENTO_TIPO_NOMBRE']).sum().reset_index()

df_pivot = df.pivot(index=cols_unpivot, 
                      columns=['MOVIMIENTO_TIPO_NOMBRE'], 
                      values='SUBS').reset_index().rename_axis(None, axis=1)
                      
df_pivot = df_pivot.fillna(0)

In [18]:
df_pivot_gf = df_pivot


In [19]:
df_pivot_gf['CHURN TOTAL'] = df_pivot_gf['CHURN CARTERA'] + df_pivot_gf['CHURN FRAUDE'] + df_pivot_gf['CHURN VOLUNTARIO'] + df_pivot_gf['CHURN MIGRACION'] + df_pivot_gf['CHURN PORTACION']


In [20]:
df_pivot_gf.head()

,PERIODO_PROCESO_CODIGO,MOVIMIENTO_ORIGEN,ANTIGUEDAD_AGRUP,ANTIGUEDAD_MESES,PLAN_TIPO_COMERCIAL,PLAN_NOMBRE_H,PLAN_NOMBRE_NH,CONSULTOR_CANAL_NOMBRE,CONSULTOR_CANAL_NOMBRE_H,CATEGORIA_BOUND,OPERADOR_PORTA_ORIGEN,REGIONAL,CHURN CARTERA,CHURN FRAUDE,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,GROSS,PERMANECE,CHURN TOTAL
0,202301,GROSS ADDS MIGRACION,ANTIGUO,10,B2B,S,Plan B2B S 18_GB/ili_MIN/ili_SMS,Tiendas,Shops,OTROS,LINEA NUEVA,Regional Oriente,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,M,Internet Móvil M3 30_GB/full_VEL,Tiendas,Shops,OTROS,LINEA NUEVA,Regional Costa,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,M,Internet Móvil M3 30_GB/full_VEL,Tiendas,Shops,OTROS,LINEA NUEVA,Regional Occidente,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,XL,#Internet Móvil XL ili_GB/full_VEL,Inbound,Inbound,Inbound,LINEA NUEVA,Regional Centro,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,XL,#Internet Móvil XL ili_GB/full_VEL,Outbound,Outbound,Outbound,LINEA NUEVA,Regional Centro,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


In [21]:
df_pivot_gf['ENDING'] = np.where(df_pivot_gf['ANTIGUEDAD_MESES'] == '0', df_pivot_gf['GROSS'] - (df_pivot_gf['CHURN CARTERA'] +
                        df_pivot_gf['CHURN FRAUDE'] + 
                        df_pivot_gf['CHURN MIGRACION'] + df_pivot_gf['CHURN PORTACION'] +
                        df_pivot_gf['CHURN VOLUNTARIO']), df_pivot_gf['PERMANECE'])

In [22]:
df_pivot_gf.groupby(['PERIODO_PROCESO_CODIGO']).sum()['ENDING']

PERIODO_PROCESO_CODIGO
202301    1385916.0
202302    1435491.0
202303    1488458.0
Name: ENDING, dtype: float64

In [23]:
df_pivot_gf[df_pivot_gf['ANTIGUEDAD_MESES'] == '0'].groupby(['PERIODO_PROCESO_CODIGO']).sum()['CHURN TOTAL']

PERIODO_PROCESO_CODIGO
202301    2636.0
202302    2606.0
202303    2969.0
Name: CHURN TOTAL, dtype: float64

In [24]:
#df_base_composition

(df_pivot_gf[df_pivot_gf['PERIODO_PROCESO_CODIGO']==mes_fin].groupby('PLAN_NOMBRE_NH').sum()['PERMANECE']/\
    sum(df_pivot_gf[df_pivot_gf['PERIODO_PROCESO_CODIGO']==mes_fin].groupby('PLAN_NOMBRE_NH').sum()['PERMANECE'])*100)

PLAN_NOMBRE_NH
#Int Móvil Plus M 90_GB/full_VEL/65_COM    0.002338
#Int Móvil Plus XL ilim_GB/full_VEL        0.000377
#Int Móvil Plus XL ilimi_GB/full_VEL       0.000754
#Internet Móvil L 80_GB/full_VEL           0.002338
#Internet Móvil L 80_GB/full_VEL_          0.000528
                                             ...   
Plan XL ili_GB/ili_MIN/ili_SMS             1.166051
Plan XL ilim_GB/ilim_Min y SMS             0.507795
Plan XL3 ili_GB/ili_MIN/ili_SMS            3.362324
Plan XS 15_GB/ili_MIN/ili_SMS              1.503023
Plan XS 18_GB/ilim_Min y SMS               0.389312
Name: PERMANECE, Length: 126, dtype: float64

In [25]:
#masks
mask_antiguedad = df_pivot_gf['ANTIGUEDAD_MESES'] == '0'
mask_periodo = df_pivot_gf['PERIODO_PROCESO_CODIGO'].isin([202209,202208,202209,202210,202211,202212,202301,202302,202303])

In [26]:
df_churn = df_pivot_gf.loc[mask_periodo].groupby(['PERIODO_PROCESO_CODIGO', 'MOVIMIENTO_ORIGEN', 'ANTIGUEDAD_AGRUP', 'ANTIGUEDAD_MESES', 'PLAN_TIPO_COMERCIAL', 'PLAN_NOMBRE_H','PLAN_NOMBRE_NH', 'CONSULTOR_CANAL_NOMBRE','CONSULTOR_CANAL_NOMBRE_H', 'CATEGORIA_BOUND',
              'OPERADOR_PORTA_ORIGEN', 'REGIONAL']).sum().reset_index()

In [27]:
df_churn['CHURN TOTAL'] = df_churn['CHURN CARTERA'] + df_churn['CHURN FRAUDE'] + df_churn['CHURN VOLUNTARIO'] + df_churn['CHURN MIGRACION'] + df_churn['CHURN PORTACION']
df_churn['OPENING'] = np.where(df_churn['ANTIGUEDAD_MESES'] != '0', df_churn['CHURN TOTAL'] + df_churn['PERMANECE'],0)

In [28]:
plan_churn_share = (df_churn.groupby('PLAN_NOMBRE_NH').sum()['CHURN TOTAL']/\
    sum(df_churn.groupby('PLAN_NOMBRE_NH').sum()['CHURN TOTAL'])).to_dict()

In [29]:
df_churn

,PERIODO_PROCESO_CODIGO,MOVIMIENTO_ORIGEN,ANTIGUEDAD_AGRUP,ANTIGUEDAD_MESES,PLAN_TIPO_COMERCIAL,PLAN_NOMBRE_H,PLAN_NOMBRE_NH,CONSULTOR_CANAL_NOMBRE,CONSULTOR_CANAL_NOMBRE_H,CATEGORIA_BOUND,...,CHURN CARTERA,CHURN FRAUDE,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,GROSS,PERMANECE,CHURN TOTAL,ENDING,OPENING
0,202301,GROSS ADDS MIGRACION,ANTIGUO,10,B2B,S,Plan B2B S 18_GB/ili_MIN/ili_SMS,Tiendas,Shops,OTROS,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,3.0
1,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,M,Internet Móvil M3 30_GB/full_VEL,Tiendas,Shops,OTROS,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,M,Internet Móvil M3 30_GB/full_VEL,Tiendas,Shops,OTROS,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
3,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,XL,#Internet Móvil XL ili_GB/full_VEL,Inbound,Inbound,Inbound,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,202301,GROSS ADDS MIGRACION,ANTIGUO,10,BroadBand,XL,#Internet Móvil XL ili_GB/full_VEL,Outbound,Outbound,Outbound,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88689,202303,GROSS ADDS PORTACION,NUEVO,3,Mobile,XS,Plan XS 15_GB/ili_MIN/ili_SMS,Outbound,Outbound,Outbound,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0
88690,202303,GROSS ADDS PORTACION,NUEVO,3,Mobile,XS,Plan XS 15_GB/ili_MIN/ili_SMS,Outbound,Outbound,Outbound,...,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0,4.0,5.0
88691,202303,GROSS ADDS PORTACION,NUEVO,3,Mobile,XS,Plan XS 15_GB/ili_MIN/ili_SMS,Tiendas,Shops,OTROS,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
88692,202303,GROSS ADDS PORTACION,NUEVO,3,Mobile,XS,Plan XS 15_GB/ili_MIN/ili_SMS,Tiendas,Shops,OTROS,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0


In [30]:
df_churn.to_excel('CHURN MARZO3.xlsx')